## Input Analysis

In [97]:
import numpy as np
import scipy.stats as st
import pandas as pd
import datetime as dt


data = pd.read_csv('TonerItDown.csv')
data = data[data['Time of day']<24]
n = len(data)

#Create rate table for Request generation
emp_rate = data.groupby(data['Time of day'].apply(np.floor)).size() / 60
emp_rate = emp_rate.values
coeffs = np.polyfit(np.arange(3,22),emp_rate[3:22],deg = 2)
fitted_rate = np.zeros(24)
fitted_rate[3:22] = coeffs[0]*np.arange(3,22) ** 2 + coeffs[1]*np.arange(3,22) + coeffs[2]
fitted_rate[[0,1,2,22,23]] = np.mean(emp_rate[[0,1,2,22,23]])
rates = pd.DataFrame(data = fitted_rate,columns = ['fitted rate'])


#Paramaters for Initial Diagnosis Time fitted to the normal distribution 
param_diagnose_a = st.norm.fit(data[data['Request location'].isin
                         (['BC_1','BC_10','BC_4','BC_5','BC_6','BC_7','BC_8'])]['Initial diagnose time'])
param_diagnose_b = st.norm.fit(data[data['Request location'].isin(['BC_2','BC_3','BC_9'])]
                               ['Initial diagnose time'])


# Paramaters for the beta distribution fit for Onsite Repair Time
data_on_site = data['On-site repair time'][data['Needs Replacement?'] != 'yes']
n_on_site = len(data_on_site)
params_repair_beta = st.beta.fit(data_on_site)


#Empirical Probabilities of a call orginating at a BC
p_hat = data.groupby(by = 'Request location')['Initial diagnose time'].size() / n
std = np.sqrt(p_hat * (1-p_hat))
probabilities = p_hat.values.tolist()


# Create Python objects from data given in problem specification
Distances = {
            "BC1":{"BC2": 20,"BC3": 30,"BC4": 45,"BC5": 50,"BC6": 50,"BC7": 60,"BC8": 55,"BC9": 60,"BC10": 70,"Dispatch":45},
            "BC2":{"BC1": 20,"BC3": 10,"BC4": 25,"BC5": 30,"BC6": 50,"BC7": 60,"BC8": 55,"BC9": 60,"BC10": 70,"Dispatch": 45},
            "BC3":{"BC1": 30,"BC2": 10,"BC4": 15,"BC5": 20,"BC6": 40,"BC7": 50,"BC8": 45,"BC9": 50,"BC10": 60,"Dispatch": 35},
            "BC4":{"BC1": 45,"BC2": 25,"BC3": 15,"BC5": 5 ,"BC6": 55,"BC7": 65,"BC8": 60,"BC9": 65,"BC10": 75,"Dispatch": 50},
            "BC5":{"BC1": 50,"BC2": 30,"BC3": 20,"BC4": 5 ,"BC6": 60,"BC7": 70,"BC8": 65,"BC9": 70,"BC10": 80,"Dispatch": 55},
            "BC6":{"BC1": 50,"BC2": 50,"BC3": 40,"BC4": 55,"BC5": 60,"BC7": 10,"BC8": 5 ,"BC9": 10,"BC10": 20,"Dispatch": 25},
            "BC7":{"BC1": 60,"BC2": 60,"BC3": 50,"BC4": 65,"BC5": 70,"BC6": 10,"BC8": 15,"BC9": 20,"BC10": 10,"Dispatch": 35},
            "BC8":{"BC1": 55,"BC2": 55,"BC3": 45,"BC4": 60,"BC5": 65,"BC6": 5 ,"BC7": 15,"BC9": 5 ,"BC10": 15,"Dispatch": 30},
            "BC9":{"BC1": 60,"BC2": 60,"BC3": 50,"BC4": 65,"BC5": 70,"BC6": 10,"BC7": 20,"BC8": 5 ,"BC10": 10,"Dispatch": 35},
            "BC10":{"BC1":70,"BC2": 70,"BC3": 60,"BC4": 75,"BC5": 80,"BC6": 20,"BC7": 10,"BC8": 15,"BC9" : 10,"Dispatch": 45}}

BusinessCenters = ["BC1", "BC2", "BC3", "BC4", "BC5", "BC6", "BC7", "BC8", "BC9", "BC10"]

BC_probabilities = (("BC1", 0.039), ("BC2", 0.082), ("BC3", 0.108), ("BC4", 0.135),
                    ("BC5", 0.118), ("BC6", 0.055), ("BC7", 0.124), ("BC8", 0.058),
                    ("BC9", 0.137), ("BC10", 0.142))

In [240]:
class Request:
    def __init__(self, initialized_time):
        self.location = np.random.choice(BusinessCenters,1,p=probabilities)[0]
        self.status = "Waiting"
        self.initialized_time = None
        self.times = []
        self.assigned_mechanic = None
        self.assigned_van = None
        self.first_req(initialized_time)
    
    #Creates random interarrival time for the request to be created, appends time to times
    def first_req(self,initialized_time):
        tmp = initialized_time + dt.timedelta(minutes=int(np.random.poisson((rates[rates.index == 0]['fitted rate'] * 60), 1)[0]))
        self.initialized_time = tmp
        self.times.append(tmp)
        
    #Check if this repair can be done on-site and update the status accordingly
    def update_waiting_status(self):
        if self.status == "Waiting":
            self.status = np.random.choice(["Onsite","Replace"],1,p=[.4,.6])[0] # fix these probs
    
    # Mechanic is found in sim_main. Assigns the found mechanic to the request
    def assign_mechanic (self, mechanicid) :
        self.assigned_mechanic = mechanicid     
       
    #function that calculates the time required to travel between the customers location and [loc]
    def add_travel_time(self, now, loc) :
        self.times.append(now + dt.timedelta(minutes=Distances[self.location][loc]/60))
        
    # Generate Time for mechanic to diagnose the problem
    def gen_diagnose_time(self,now):
        if self.location in ['BC_2','BC_3','BC_9']:
            self.times.append(now + dt.timedelta(minutes=np.random.normal(param_diagnose_b[0], param_diagnose_b[1])))
        else: 
            self.times.append(now + dt.timedelta(minutes=np.random.normal(param_diagnose_a[0], param_diagnose_a[1])))
    
    def gen_onsite_repair_time(self,now):
        self.times.append(now + dt.timedelta(minutes = np.random.beta(params_repair_beta[0], 
                                            params_repair_beta[1])))
    def gen_van_swap_time(self, now) :
        self.times.append(now + dt.timedelta(minutes= np.random.triangular(10,15,25)))
        
    #Random time for a van to replace the copier
    def gen_van_replace_time(self, now) :
        self.times.append(now + dt.timedelta(minutes= np.random.triangular(20,30,60)))
    
    #Van if found in sim_main. Assigns the found vanid to the request
    def assign_van (self,vanid) :
        self.assigned_van = vanid

        
class Van:
    def __init__(self,id):
        self.loc = 'Dispatch'
        self.busy = False
        self.id = id
        
        
class Mechanic:
    def __init__(self,id):
        self.loc = 'Dispatch'
        self.busy = False
        self.id = id

In [250]:
class Simulation:
    def __init__(self, n_mechanics, n_vans):
        
        self.n_mechanics = n_mechanics
        self.n_vans = n_vans
        self.n_runs = n_runs
        self.mechanics = []
        self.vans = []
        self.open_requests = []
        self.closed_requests = []
        
        for m in range(1,self.n_mechanics):
            self.mechanics.append(Mechanic(m))
        for v in range(1,self.n_vans):
            self.vans.append(Van(v))
        current_time = dt.datetime.combine(dt.date.today(), dt.time(hour=0))
        end_time = dt.datetime.combine(dt.date.today(), dt.time(hour=23))
        first_request = Request(current_time) 
        self.open_requests.append(first_request)
        while(current_time < end_time):
            update_num = len((self.open_requests[0]).times)                # Current step for the request
            if(update_num == 1):                                           # We need to move a mechanic
                mech = self.find_nearest("Mechanic")                       # Find nearest free mechanic
                mech.busy = True                                           # Mechanic is now occupied
                self.open_requests[0].assign_mechanic(mech.id)             # Give the request an assigned mechanic
                self.open_requests[0].add_travel_time(current_time, mech.loc)  # Add the mechanics travel time
                self.open_requests.insert(0,Request(current_time))         # Create a new next request
                self.move_first()
                self.move_first()
            elif(update_num == 2):                                         # We need to diagnose the problem
                self.open_requests[0].gen_diagnose_time(current_time)      # Randomly generate diagnosis time
                self.open_requests[0].update_waiting_status()              # Randomly generate repair / replace
                self.move_first()
            elif(update_num == 3):                                         # Need to either repair or call van 
                if(self.open_requests[0].status == "Onsite"):              # It's a repair
                    self.open_requests[0].gen_onsite_repair_time(current_time) # Generate a repair time
                else:                                                      # Need to request a van
                    van = self.find_nearest("Van")    # Find a van
                    van.busy = True
                    self.open_requests[0].assign_van(van.id)               # Assign the van to the request
                    self.open_requests[0].add_travel_time(current_time, van.loc) # Add travel time to site
                    self.free_worker("Mechanic")                           # Mechanic is now done with task
                self.move_first()
            elif(update_num == 4):                                         # Finish off the van replacement of  
                if(self.open_requests[0].status == "Onsite"): 
                    self.free_worker("Mechanic")                           # Worker finished repair, now free
                    tmp_request = self.open_requests.pop(0)                # Remove the current request
                    self.closed_requests.append(tmp_request)               # Move to the end of the list, done with it
                else:
                    self.open_requests[0].gen_van_replace_time(current_time)
                    self.move_first()
            elif(update_num == 5): # Wait for van to finish 
                self.open_requests[0].add_travel_time(current_time, 'Dispatch')
                self.move_first()
            elif(update_num == 6):
                self.open_requests[0].gen_van_swap_time(current_time)
                self.move_first()
            elif(update_num == 7):
                tmp = self.open_requests.pop(0)
                self.closed_requests.append(tmp)
                self.free_worker("Van")
            current_time = self.open_requests[0].times[len(self.open_requests[0].times) - 1]
            
            
            
    # Find the nearest free worker (of given type) of out of list and set to busy
    def find_nearest(self, worker_type):
        closest_distance = 100
        assigned_worker = None
        request_loc = self.open_requests[0].location      #location of the request
        
        loop_list = self.mechanics if (worker_type == "Mechanics") else self.vans
       
        for worker in loop_list:
            if(worker.busy == True): pass
            elif(Distances[request_loc][worker.loc] < closest_distance):
                assigned_worker = worker
                closest_distance = Distances[request_loc][worker.loc]
                
        return assigned_worker
        

    def free_worker(self, worker_type):
        if(worker_type == "Mechanic"):
            for mechanic in self.mechanics:
                if(mechanic.id == self.open_requests[0].assigned_mechanic): mechanic.busy = False
                    
        elif(worker_type == "Van"):
            for van in self.vans:
                if(van.id == self.open_requests[0].assigned_van): van.busy = False

    def move_first(self):
        if(len(self.open_requests) > 1):
            tmp_req = self.open_requests.pop(0)
            index = 0
            try: 
                while(self.open_requests[index].times[-1] < tmp_req.times[-1]): index += 1
                self.open_requests.insert(index-1, tmp_req)
            except(IndexError):
                self.open_requests.append(tmp_req)
  

In [319]:
n_runs = 10
max_mechanics = 51
max_vans = 51
results = pd.DataFrame()
# Run simulation n times, storing the results in a df
for m in range(40,max_mechanics):
    for v in range(40, max_vans):
        for x in range(1, n_runs):
            sim = Simulation(m, v)
            df = pd.DataFrame([y.times for y in sim.closed_requests])
            df['status'] = pd.DataFrame([y.status for y in sim.closed_requests])
            df['Run'] = x
            df['n_Mechanics'] = m
            df['n_Vans'] = v
            df['Response'] = df[1] - df[0]
            df['Replace'] = np.where(df['status'] == 'Replace', df[3] - df[1], None)
            results = results.append(df)  
        results.to_csv("~/School/ORIE4580/ORIE4580/data/" + str(m) + ", " + str(v) + ".csv")

In [321]:
pd.read_csv("~/School/ORIE4580/ORIE4580/data/40, 40.csv")

Unnamed: 0                           0                           1  \
0             0  2019-11-29 01:06:00.000000  2019-11-29 00:00:50.000000   
1             1  2019-11-29 01:01:00.000000  2019-11-29 01:01:35.000000   
2             2  2019-11-29 02:09:00.000000  2019-11-29 02:09:35.000000   
3             3  2019-11-29 03:08:00.000000  2019-11-29 03:08:25.000000   
4             4  2019-11-29 04:18:00.000000  2019-11-29 04:18:45.000000   
..          ...                         ...                         ...   
175          16  2019-11-29 17:22:00.000000  2019-11-29 17:22:50.000000   
176          17  2019-11-29 18:19:00.000000  2019-11-29 18:19:55.000000   
177          18  2019-11-29 19:30:00.000000  2019-11-29 19:30:30.000000   
178          19  2019-11-29 20:35:00.000000  2019-11-29 20:35:55.000000   
179          20  2019-11-29 21:48:00.000000  2019-11-29 21:48:45.000000   

                              2                           3  \
0    2019-11-29 00:15:51.495865  2019-11-29 00:15:55.273588   
1    2019-11-29 01:16:50.731654  2019-11-29 01:17:25.731654   
2    2019-11-29 02:28:16.694676  2019-11-29 02:28:51.694676   
3    2019-11-29 03:24:21.695865  2019-11-29 03:24:47.612982   
4    2019-11-29 04:33:06.629200  2019-11-29 04:33:51.629200   
..                          ...                         ...   
175  2019-11-29 17:40:18.411905  2019-11-29 17:40:28.545984   
176  2019-11-29 18:36:02.170629  2019-11-29 18:36:57.170629   
177  2019-11-29 19:51:49.891031  2019-11-29 19:52:19.891031   
178  2019-11-29 20:49:32.412210  2019-11-29 20:50:27.412210   
179  2019-11-29 22:02:05.530782  2019-11-29 22:02:50.530782   

                              4                           5  \
0                           NaN                         NaN   
1    2019-11-29 02:13:20.732854  2019-11-29 02:13:55.732854   
2    2019-11-29 02:59:32.301583  2019-11-29 03:00:07.301583   
3                           NaN                         NaN   
4    2019-11-29 05:09:21.118744  2019-11-29 05:10:06.118744   
..                          ...                         ...   
175                         NaN                         NaN   
176  2019-11-29 19:28:13.776480  2019-11-29 19:29:08.776480   
177  2019-11-29 20:33:38.833504  2019-11-29 20:34:08.833504   
178  2019-11-29 21:31:36.301862  2019-11-29 21:32:31.301862   
179  2019-11-29 22:36:46.766693  2019-11-29 22:37:31.766693   

                              6   status  Run  n_Mechanics  n_Vans  \
0                           NaN   Onsite    1           40      40   
1    2019-11-29 02:25:24.473179  Replace    1           40      40   
2    2019-11-29 03:12:55.932031  Replace    1           40      40   
3                           NaN   Onsite    1           40      40   
4    2019-11-29 05:27:24.726123  Replace    1           40      40   
..                          ...      ...  ...          ...     ...   
175                         NaN   Onsite    9           40      40   
176  2019-11-29 19:41:31.070140  Replace    9           40      40   
177  2019-11-29 20:54:03.608785  Replace    9           40      40   
178  2019-11-29 21:49:50.155021  Replace    9           40      40   
179  2019-11-29 22:49:22.259994  Replace    9           40      40   

                        Response       Replace  
0    -1 days +22:54:50.000000000           NaN  
1      0 days 00:00:35.000000000  9.507317e+11  
2      0 days 00:00:35.000000000  1.156695e+12  
3      0 days 00:00:25.000000000           NaN  
4      0 days 00:00:45.000000000  9.066292e+11  
..                           ...           ...  
175    0 days 00:00:50.000000000           NaN  
176    0 days 00:00:55.000000000  1.022171e+12  
177    0 days 00:00:30.000000000  1.309891e+12  
178    0 days 00:00:55.000000000  8.724122e+11  
179    0 days 00:00:45.000000000  8.455308e+11  

[180 rows x 14 columns]